In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt
from datetime import datetime
from scipy.stats import linregress
from citipy import citipy



In [4]:
visitor_path = "Data/changes-visitors-covid.csv"
covid_path = "Data/covid-data.csv"
debt_path = "Data/debt-relief-covid.csv"
income_path = "Data/income-support-covid.csv"
face_path = "Data/face-covering-policies-covid.csv"
travel_path = "Data/international-travel-covid.csv"
school_path = "Data/school-closures-covid.csv"


In [11]:
visitor_data = pd.read_csv(visitor_path)
covid_data = pd.read_csv(covid_path)
debt_data = pd.read_csv(debt_path)
income_data = pd.read_csv(income_path)
face_data = pd.read_csv(face_path)
travel_data = pd.read_csv(travel_path)
school_data = pd.read_csv(school_path)


In [12]:
visitor_data.head()

,Entity,Code,Day,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,Afghanistan,AFG,2020-02-17,0.00,4.000,1.333,5.667,1.000,-6.333
1,Afghanistan,AFG,2020-02-18,0.75,4.250,0.750,6.500,1.500,-3.250
2,Afghanistan,AFG,2020-02-19,0.40,4.000,0.800,5.200,1.400,-1.600
3,Afghanistan,AFG,2020-02-20,0.00,3.833,0.833,5.000,1.000,-0.333
4,Afghanistan,AFG,2020-02-21,0.00,4.571,0.714,5.286,1.429,0.571


In [135]:
covid_data.location.nunique()

230

In [136]:
covid_data.location.unique().tolist()

['Afghanistan',
 'Africa',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Europe',
 'European Union',


In [137]:
continent = ['North America','South America','Africa','Asia','Europe']

In [140]:
# removing continents from data frame
covid_data[~covid_data.location.isin(continent)]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96203,ZWE,Africa,Zimbabwe,2021-06-12,39852.0,164.0,97.714,1632.0,3.0,3.857,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96204,ZWE,Africa,Zimbabwe,2021-06-13,39959.0,107.0,110.000,1632.0,0.0,3.714,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96205,ZWE,Africa,Zimbabwe,2021-06-14,40077.0,118.0,119.857,1635.0,3.0,3.429,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN
96206,ZWE,Africa,Zimbabwe,2021-06-15,40318.0,241.0,142.429,1637.0,2.0,2.857,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN


In [110]:
covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [124]:
covid_data.head()
covid_df = round(covid_data.groupby(["location"])["total_cases"].max(),2)
# type(covid_df)
covid_df.dropna(inplace=True)
covid_df

location
Afghanistan        93288.0
Africa           5101570.0
Albania           132476.0
Algeria           134458.0
Andorra            13836.0
                  ...     
Vietnam            11794.0
World          177040182.0
Yemen               6867.0
Zambia            118850.0
Zimbabwe           40556.0
Name: total_cases, Length: 201, dtype: float64

In [130]:
covid_data[covid_data.location == "Afghanistan"].describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
count,479.000000,479.000000,474.000000,452.000000,452.000000,474.000000,479.000000,479.000000,474.000000,452.000000,...,0.0,4.790000e+02,4.790000e+02,0.0,0.0,4.790000e+02,479.0,4.790000e+02,4.790000e+02,0.0
mean,38627.185804,194.755741,189.940300,1610.030973,8.148230,7.450859,992.263860,5.002954,4.879247,41.358856,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
std,21882.369173,302.593270,273.168902,932.423675,11.198577,8.843587,562.119206,7.773092,7.017222,23.952284,...,NaN,1.820891e-12,6.401570e-14,NaN,NaN,4.054328e-13,0.0,7.966399e-13,2.889598e-15,NaN
min,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.026000,0.000000,0.000000,0.026000,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
25%,29309.000000,28.500000,39.785500,1033.000000,1.000000,1.714000,752.896500,0.732000,1.022000,26.536000,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
50%,40369.000000,78.000000,88.571500,1545.500000,4.000000,4.286000,1037.008000,2.004000,2.275500,39.701500,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
75%,55529.000000,214.500000,208.321000,2435.250000,11.000000,9.964250,1426.441500,5.510500,5.351500,62.557500,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN
max,93288.000000,1842.000000,1584.000000,3683.000000,85.000000,61.714000,2396.403000,47.318000,40.690000,94.610000,...,NaN,5.970290e+02,9.590000e+00,NaN,NaN,3.774600e+01,0.5,6.483000e+01,5.110000e-01,NaN


In [134]:
covid_data[covid_data.total_cases == 93288.000000]


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
478,AFG,Asia,Afghanistan,2021-06-16,93288.0,16.0,1319.714,3683.0,0.0,54.0,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
15685,CAN,North America,Canada,2020-06-01,93288.0,809.0,881.286,7952.0,46.0,76.0,...,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929,NaN


In [99]:
len(covid_data)

96208

In [49]:
debt_data.head()

,Entity,Code,Day,debt_relief
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [83]:
income_data.head()


,Entity,Code,Day,income_support
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [97]:

income_df = round(income_data.groupby(["Entity"])["income_support"].mean(),2)
income_df
income_df.sort_values(ascending = False)

Entity
Sweden         1.74
Belgium        1.73
Andorra        1.73
France         1.72
Denmark        1.72
               ... 
Lebanon        0.00
Ethiopia       0.00
Zambia         0.00
Ghana          0.00
Afghanistan    0.00
Name: income_support, Length: 185, dtype: float64

In [75]:
df = income_data.set_index("Entity")

# df = income_data.loc["Afghanistan"]
df_income = df.loc["Bulgaria"]
df_income["income_support"].sort_values(ascending = False)

Entity
Bulgaria    2
Bulgaria    2
Bulgaria    2
Bulgaria    2
Bulgaria    2
           ..
Bulgaria    0
Bulgaria    0
Bulgaria    0
Bulgaria    0
Bulgaria    0
Name: income_support, Length: 517, dtype: int64

In [16]:
face_data.head()

,Entity,Code,Day,facial_coverings
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [17]:
travel_data.head()

,Entity,Code,Day,international_travel_controls
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [20]:
school_data.head()


,Entity,Code,Day,school_closures
0,Afghanistan,AFG,2020-01-21,0
1,Afghanistan,AFG,2020-01-22,0
2,Afghanistan,AFG,2020-01-23,0
3,Afghanistan,AFG,2020-01-24,0
4,Afghanistan,AFG,2020-01-25,0
